# Demo
This was my process of selecting a scoring method to determine which model evaluates our code 

## Setup stuff

In [39]:
import langchain
import ollama

In [40]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    device_map=None,
    torch_dtype="float32",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    trust_remote_code=True
)


In [41]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model2 = AutoModelForCausalLM.from_pretrained(
    "Eyas1/fine_tuned_codellama_java_docs",
    device_map=None,
    torch_dtype="float32",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Eyas1/fine_tuned_codellama_java_docs",
    trust_remote_code=True
)


In [42]:
from langchain.prompts import PromptTemplate

multiShot = PromptTemplate(
    input_variables=["method_code"],
    template='''\
Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** 

Example 1:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Javadoc:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

Example 2:
Method:
private int add(int a, int b) {{
    return a + b;
}}

Javadoc:
/**
 * Adds two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the sum of the two integers
 */

Now write a Javadoc comment for this method:
Method:
{method_code}


'''
)

# Test methods

In [43]:
longMethod = """private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
"""

In [44]:
longmethod2 = '''
public BallSeq clone( )
    {{
        assert wellFormed() : "invariant wrong at start of clone()";

        BallSeq result;

        try
        {{
            result = (BallSeq) super.clone( );
        }}
        catch (CloneNotSupportedException e)
        {{
            // This exception should not occur. But if it does, it would probably
            // indicate a programming error that made super.clone unavailable.
            // The most common error would be forgetting the "Implements Cloneable"
            // clause at the start of this class.
            throw new RuntimeException
            ("This class does not implement Cloneable");
        }}
        // This is inspired by the textbook for this class, page 203
        if(result.head == null) return result;

        Node source = result.head;
        Node copyHead = new Node(source.data, null);
        Node copyTail = copyHead;

        while(source.next != null) {{
            if(source == cursor) result.cursor = copyTail;
            if(source == precursor) result.precursor = copyTail;
            if(source == tail) result.tail = source;
            source = source.next;
            copyTail.next = new Node(source.data, null);
            copyTail = copyTail.next;
        }}
        result.head = copyHead;
        if(source == cursor) result.cursor = copyTail;
        if(source == tail) result.tail = copyTail;

        // TODO: Implemented by student.
        // Now do the hard work of cloning the list.
        // See pp 200-204, 235 (3rd ed. pp. 193-197, 228)
        // Setting precursor, cursor and tail correctly is tricky.


        assert wellFormed() : "invariant wrong at end of clone() for this";
        assert result.wellFormed() : "invariant wrong for result of clone() for copy";
        return result;
    }}
'''

In [45]:
shortMethod = """
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
 """

In [46]:
def score_method_complexity(method_code):
    lines = method_code.strip().splitlines()
    score = 0

    # Base score: length of method
    if len(lines) > 20:
        score += 2
    elif len(lines) > 10:
        score += 1

    # Keywords that indicate control flow or complexity
    keywords = ["for", "while", "if", "else", "switch", "case", "try", "catch", "finally", "return"]
    score += sum(1 for line in lines for kw in keywords if kw in line)

    # Additional indicators of advanced logic
    if "clone" in method_code or "recursion" in method_code.lower():
        score += 2
    if "assert" in method_code:
        score += 1
    if "TODO" in method_code or "textbook" in method_code.lower():
        score += 1

    # Thresholds
    if score >= 8:
        return "super_difficult", score
    elif score >= 4:
        return "complex", score
    else:
        return "simple", score


## Testing Scoring System 

In [47]:
score_method_complexity(longMethod)

('super_difficult', 29)

In [48]:
score_method_complexity(shortMethod)

('simple', 3)

In [49]:
from langchain.prompts import PromptTemplate

routePrompt = PromptTemplate(input_variables=["method_code"],
    template='''
    You are a helpful assistant that helps to determine the complexity of a method.
    You are given a method and you need to determine if it is simple, complex or super difficult.
    A method is considered simple if it has a low number of lines and control flow statements.
    A method is considered complex if it has a moderate number of lines and control flow statements.
    A method is considered super difficult if it has a high number of lines and control flow statements.


    you are only allowed to respond in one word simple, complex or super_difficult.
    You are given the following method:
    {method_code}
    '''
)

## testing outputs

In [50]:
from ollama import chat
response = chat(model=qwenSmall, messages=[{'role': 'user', 'content': routePrompt.format(method_code=shortMethod)}])
print(response['message']['content'])

NameError: name 'qwenSmall' is not defined

In [ ]:
from ollama import chat
response = chat(model=qwenSmall, messages=[{'role': 'user', 'content': routePrompt.format(method_code=longMethod)}])
print(response['message']['content'])

simple


# Cyclomatic complexity 
this library looks at how many outcomes our code can take the more outcomes the higher the score 

In [ ]:
# gets the cyclomatic complexity of a method
import lizard

def get_java_method_complexity(method_string):
    wrapped = f" {{\n{method_string}\n}}"
    results = lizard.analyze_file.analyze_source_code("Dummy.java", wrapped)
    complexity = results.function_list[0].cyclomatic_complexity
    return complexity

## Testing

In [ ]:
get_java_method_complexity(longMethod)

14

In [ ]:
get_java_method_complexity(shortMethod)

3

In [ ]:
get_java_method_complexity(longmethod2)

9

## Demo

In [53]:
myMethod = multiShot.format(method_code="""
your method code here""")


In [56]:
complexity = get_java_method_complexity(myMethod)


In [ ]:
selected_model = model2 if complexity > 7 else model

inputs = tokenizer(multiShot.format(method_code=myMethod), return_tensors="pt").to(selected_model.device)
outputs = selected_model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** 

Example 1:
Method:
public String reverseProcessed(String text) {
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}

Javadoc:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

Example 2:
Method:
private int add(int a, int b) {
    return a + b;
}

Javadoc:
/**
 * Adds two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the sum of the two integers
 */

Now write a Javadoc comment for this method:
Method:
Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** 

Example 1:
Method:
public String reverseProcessed(String